In [30]:
from AutoDiff import *
from numpy.testing import assert_array_equal, assert_array_almost_equal

In [7]:
def tan(x):
    try: #rAD
        ad = rAD(np.tan(x.val))
        x.children.append((1/(np.cos(x.val)**2),ad))
        return ad
    except AttributeError:
        try: #fAD
            return fAD(np.tan(x.val), x.der/(np.cos(x.val)**2))
        except AttributeError:
            return np.tan(x) #numeric

In [35]:
def test_combined_tan():
    x = fAD(5)
    y = tan(x**2)    
    a = rAD(5)
    b = tan(a**2)
    b.outer()
    a.grad()
    assert(y.val[0] == np.tan(25))
    assert(y.der[0][0] == 10/np.cos(25)**2)
    assert(b.val == np.tan(25))
    assert_array_almost_equal(a.der,10/np.cos(25)**2)    
test_combined_tan()

In [36]:
def cosh(x):
    try:
        #if x is an rAD object
        new = rAD(np.cosh(x.val)) #
        x.children.append((np.sinh(x.val), new))
        return new
    except AttributeError:
        try:
            #if x is an fAD object
            return fAD(np.cosh(x.val), np.sinh(x.val)*x.der)
        except AttributeError:
            #if x is a number
            return np.cosh(x)       

In [53]:
def test_combined_cosh():
    x = fAD(8)
    y = cosh(x) + x
    a = rAD(8)
    b = cosh(a) - a
    b.outer()
    a.grad()
    assert y.val == np.cosh(8)+8
    assert y.der == np.sinh(8)+1
    assert b.val == np.cosh(8)-8
    assert a.der == np.sinh(8)-1
test_combined_cosh()

In [65]:
def tanh(x):
    try:
        #if x is an rAD object
        new = rAD(np.tanh(x.val))
        x.children.append((1/(np.cosh(x.val)**2),new))
        return new
    except AttributeError:
        try:
            #if x is an fAD object
            return fAD(np.tanh(x.val), (x.der/(np.cosh(x.val)**2)))
        except AttributeError:
            return np.tanh(x)

In [70]:
def test_combined_tanh():
    x = fAD(3)
    y = tanh(x**2)+x**2
    a = rAD(3)
    b = tanh(a**2)+a**2
    b.outer()
    a.grad()
    assert y.val == np.tanh(9)+9
    assert y.der == 6/(np.cosh(9)**2)+6
    assert b.val == np.tanh(9)+9
    assert a.der == 6/(np.cosh(9)**2)+6
test_combined_tanh()

In [71]:
def sqrt(self):
    try: # reverse
        ad = rAD(self.val**0.5)
        self.children.append(((self.val**(-0.5))*0.5,ad))
        return ad
    except AttributeError:
        try: # forward
            return fAD(self.val**0.5, 0.5*self.der*(self.val**(-0.5)))
        except AttributeError:
            return self**0.5 #just a value            

In [90]:
def test_combined_sqrt():    
    x = fAD(6)
    y = 4*sqrt(x)
    a = rAD(6)
    b = 4*sqrt(a)
    b.outer()
    a.grad()
    assert y.val == 4*6**0.5
    assert_array_almost_equal(y.der,2*6**-0.5)
    assert b.val == 4*6**0.5
    assert_array_almost_equal(a.der,2*6**-0.5)
    
test_combined_sqrt()